# Welcome to UnicornFlix!

Congratulations! You have just been hired by UnicornFlix, which is a new Direct to Consumer Streaming service that launched and is jumping into the crowded space of Video on Demand/Free Ad Supported TV (FAST) providers. You have been hired into the Search & Discovery team which leads efforts around personalization. Currently most of your app does not provide a personalized experience, the movies are presented in a static order for all users. In order to prevent customer churn, you are looking to add personalized experiences. 

You’ve been asked by the founders to:

- Increase subscriber engagement by tailoring every experience to individual users
- Help users discover newly released content
- Increase the breadth of content offered to them from the UnicornFlix catalog
- Reduce the time to value by creating valuable recommendations in a short time

Throughout the this workshop you will be exploring your datasets, building/training several recommendation models and implementing recommendations with API's.


## In this Notebook

In this notebook, you will choose a dataset and prepare it for use with Amazon Personalize.

1. [How to Use the Notebook](#usenotebook)
1. [Introduction to Amazon Personalize Datasets](#datasets)
1. [Prepare the Item Metadata](#prepare_items)
1. [Prepare the Interactions Data](#prepare_interactions)
1. [Prepare the User Metadata](#prepare_users)


## How to Use the Notebook <a class="anchor" id="usenotebook"></a>

### Executing cells
The code is broken up into cells like the one below. There's a triangular Run button at the top of this page that you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.

Simply follow the instructions below and execute the cells to get started with Amazon Personalize.

### Understanding the code

This notebook can be used in two modalities:

1. Train as you go by executing each cell. Some cells may take a long time to finish executing as they wait for resources to be created. You will have this experience if you deployed the 'simple'  Amazon CloudFormation template ([personalizeIDSimple.yaml](../personalizeIDSimple.yaml)).

2. Go through notebook with previously created resources. All or the majority of the resources will already be created and cells will just retrieve the information of these existing resources to use them in following steps. You will have this experience if you deployed the 'pretrained' Amazon CloudFormation template ([PersonalizeIDPretrained.yaml](../PersonalizeIDPretrained.yaml)).

<div class="alert alert-block alert-warning">
<b>Note:</b> If you deployed the 'pretrained' template, you must wait at least 3 hours for all resources to tbe trained before proceeding with the workshop.
</div>

Because of this, you will find that some cells have `try` and `except` blocks. In particular most of them are handling a `ResourceAlreadyExistsException` exception. 

You can look at the code in the `try` block to get a good idea of how you can create a resource and understand how to use the Amazon Personalize SDK. The `except` block will let you know that the resource has been created and record the corresponding ARN, which is the Amazon unique identifier.

This is an example of the `try` block for creating a dataset group, this code will execute without exceptions if the dataset group does not exist and raise an exception if the dataset group does already exist:

```python
try: 
    create_dataset_group_response = personalize.create_dataset_group(
        name = workshop_dataset_group_name,
        domain='VIDEO_ON_DEMAND'
    )

    workshop_dataset_group_arn = create_dataset_group_response['datasetGroupArn']
    print(json.dumps(create_dataset_group_response, indent=2))
    print ('\nCreating the Dataset Group with dataset_group_arn = {}'.format(workshop_dataset_group_arn))
    
```
and this is the corresponding `except` block that will be exectuted if an exeption is raised becuse the dataset group already exists. This block saves the ARN for the existig dataset group to use later and lets you know the resource already exists.

```python
except personalize.exceptions.ResourceAlreadyExistsException as e:
    workshop_dataset_group_arn = 'arn:aws:personalize:'+region+':'+account_id+':dataset group/' + 
        workshop_dataset_group_name 
    print ('\nThe the Dataset Group with dataset_group_arn = {} already exists'.format(
        workshop_dataset_group_arn))
    print ('\nWe will be using the existing Dataset Group dataset_group_arn = {}'.format(
        workshop_dataset_group_arn))
```

Depending on the resource, you may also find that sometimes the code will check from a list of resourses to find if a resource exists and then use `if` and `else` blocks to either use the existing resource or create it.

### Let's build!

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK for python) and [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/)  which are core data science tools.

In [94]:
# Get the latest version of botocore to ensure we have the latest features in the SDK
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade --no-deps --force-reinstall botocore
!pip uninstall -y awscli
!pip install awscli
!pip uninstall -y numexpr
!pip install numexpr

  Using cached botocore-1.34.93-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.34.93-py3-none-any.whl (12.2 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.93
    Uninstalling botocore-1.34.93:
      Successfully uninstalled botocore-1.34.93
Found existing installation: awscli 1.32.93
Uninstalling awscli-1.32.93:
  Successfully uninstalled awscli-1.32.93
  Using cached awscli-1.32.93-py3-none-any.whl.metadata (11 kB)
Using cached awscli-1.32.93-py3-none-any.whl (4.4 MB)
Found existing installation: numexpr 2.10.0
Uninstalling numexpr-2.10.0:
  Successfully uninstalled numexpr-2.10.0
  Using cached numexpr-2.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.9 kB)
Using cached numexpr-2.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (376 kB)


In [95]:
# Import packages
import time
from time import sleep
import json
from datetime import datetime

import boto3
import pandas as pd
import numpy as np


In [96]:
# Make directories
data_dir = "poc_data"
!mkdir $data_dir
imdb_dir = data_dir+'/imdb'
!mkdir $imdb_dir

mkdir: cannot create directory ‘poc_data’: File exists
mkdir: cannot create directory ‘poc_data/imdb’: File exists


In [97]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [98]:
# Get the account id and region to use later
account_id = boto3.client('sts').get_caller_identity().get('Account')
print("account id:", account_id)

with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print("region:", region)

account id: 381491864570
region: us-east-1


If this is a workshop and the resources were created for you, we will retrieve the variables of the resources created.

In [99]:
# Opening JSON file
f = open('./params.json')
parameters = json.load(f)

In [100]:
workshop_dataset_group_name = parameters['datasetGroup']['serviceConfig']['name']

interactions_schema_name = parameters['datasets']['interactions']['schema']['serviceConfig']['name']
interactions_dataset_name = parameters['datasets']['interactions']['dataset']['serviceConfig']['name']

items_schema_name = parameters['datasets']['items']['schema']['serviceConfig']['name']
items_dataset_name = parameters['datasets']['items']['dataset']['serviceConfig']['name']

users_schema_name = parameters['datasets']['users']['schema']['serviceConfig']['name']
users_dataset_name = parameters['datasets']['users']['dataset']['serviceConfig']['name']

#The following job names are the starting Strings of the job names that can be created
interactions_import_job_name = 'dataset_import_interaction'
items_import_job_name = 'dataset_import_item'
users_import_job_name = 'dataset_import_user'

items_filename = "items.csv"
interactions_filename = "interactions.csv"
users_filename = "users.csv"

for recommender in parameters['recommenders']:
    # This is currently configured assuming only one recommender of each type, if there are multiple 
    # recommenders of the same type further configuration is needed.
    if (recommender['serviceConfig']['recipeArn'] == 'arn:aws:personalize:::recipe/aws-vod-more-like-x'):
        recommender_more_like_x_name =recommender['serviceConfig']['name'] 
    if (recommender['serviceConfig']['recipeArn'] == 'arn:aws:personalize:::recipe/aws-vod-top-picks'):
        recommender_top_picks_for_you_name =recommender['serviceConfig']['name']
        
for solution in parameters['solutions']:
    # This is currently configured assuming only one solution of this type, if there are multiple 
    # solutions of the same type further configuration is needed.
    if (solution['serviceConfig']['recipeArn'] == 'arn:aws:personalize:::recipe/aws-personalized-ranking'):
        workshop_rerank_solution_name = solution['serviceConfig']['name'] 
        # This is currently configured assuming only one campaign, if there are multiple campaigns 
        # further configuration is needed.
        workshop_rerank_campaign_name = solution['campaigns'][0]['serviceConfig']['name'] 

Make sure we can use the SDK to interact with Amazon Personalize by describing some of the pre-created resources used in the workshop. If you have not pre-deployed resources and are building them as you go with this notebook, the below cell will raise an exception. You can continue with the notebook and create resources and train models as you go.

In [101]:
try:
    # Describe a few resources using the SDK
    more_like_x_arn = 'arn:aws:personalize:'+region+':'+account_id+':recommender/'+recommender_more_like_x_name 
    describe_response = personalize.describe_recommender(recommenderArn = more_like_x_arn)

    top_picks_arn = 'arn:aws:personalize:'+region+':'+account_id+':recommender/'+recommender_top_picks_for_you_name
    describe_response = personalize.describe_recommender(recommenderArn = top_picks_arn)

    rerank_arn = 'arn:aws:personalize:'+region+':'+account_id+':solution/'+workshop_rerank_solution_name
    describe_response = personalize.describe_solution(solutionArn = rerank_arn)
    
    print("All resources have been successfully pretrained!")
    
except:
    print("Some or all pretrained resources not found. Proceed to the next cell if you will be uploading data and training models as you go.")

Some or all pretrained resources not found. Proceed to the next cell if you will be uploading data and training models as you go.


## Introduction to Amazon Personalize Datasets <a class="anchor" id="datasets"></a>
[Back to top](#top)

Regardless of the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. **UserID** - The user who interacted
1. **ItemID** - The item the user interacted with
1. **Timestamp** - The time at which the interaction occurred

In this notebook we will be importing interactions, user and item data into your environment, inspecting it and converting it to a format that will allow you use it in Amazon Personalize to train models to get personalized recommendations.

The following diagram shows the resources that we will create. with the section we are building  in this notebook highlighted in blue with a dashed outline.

![Workflow](images/01_Data_Layer_Resources.jpg)

Generally speaking your data will not arrive in a perfect form for Personalize, and will take some modification to be structured correctly. This notebook guides you through that process.

### Items data

The item data consists of information about the content that is being interacted with, this generally comes from Content Management Systems (CMS). 

<div class="alert alert-block alert-warning">
<b>Note:</b> This dataset is not manatory for Amazon Personalize to generate recommendations, but providing good item metadata will ensure the best results in your trained models.
</div>

### Item Interactions data

* Interations data: we use the ml-latest-small dataset from the [Movielens](https://grouplens.org/datasets/movielens/) project as a proxy for user-item interactions. 

The item interaction data concists of information about the interactions the users of the fictional app will have with the content. This usually comes from analytics tools or Customer Data Platform's (CDP). The best interaction data for use for Amazon Personalize would include the sequential order of user beavior, what content was watched/clicked on and the order it was interacted with. To simulate our interaction data, we will be using data from the [MovieLens project](https://grouplens.org/datasets/movielens/). Movielens offers multiple versions of their dataset, for the purposes of this workshop we will be using the reduced version of this dataset (approx 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users).

This dataset is manatory for Amazon Personalize to generate recommendations.

### User data

The user data is what information you have about you users, it usually comes from Customer relationship management (CRM) or Subscriber management systems. Since there is no user data included in the MovieLens data, we will be generating a small synthetic dataset to simulate this component of the workshop. 

<div class="alert alert-block alert-warning">
<b>Note:</b> This dataset is not manatory for Amazon Personalize to generate recommendations, but providing good item metadata will ensure the best results in your trained models.
</div>



## Prepare the Item Metadata <a class="anchor" id="prepare_items"></a>
[Back to top](#top)

In order provide additional metadata, and also to provide a consistent experience for our users we leverage a subset of the IMDb Essential Metadata for Movies/TV/OTT dataset. IMDb is the world's most popular and authoritative source for information on movies, TV shows, and celebrities and powers entertainment experiences around the world. License IMDb entertainment metadata from over 10 million movies, TV series, and Video Game titles including 12 million cast and crew, 1 billion star ratings, and global box office grosses from Box Office Mojo. All IMDb data products are updated daily and easily accessed through AWS Data Exchange. 

The IMDb Essential Metadata for Movies/TV/OTT dataset, which contains 

- 9+ million titles
- 12+ million names
- Film, TV, music and celebrities
- 1 billion ratings from the world’s largest entertainment fan community

IMDb has [multiple datasets available in the Amazon Data Exchange](https://aws.amazon.com/marketplace/seller-profile?id=0af153a3-339f-48c2-8b42-3b9fa26d3367). <img src="./images/IMDb_Logo_Rectangle.png" alt="IMDb logo" style="width:50px;"/>

For this workshop we have extracted the subset of data we needed and prepared it for use with the following information from the IMDb Essential Metadata for Movies/TV/OTT (Bulk data) dataset.

TITLE                      
YEAR                       
IMDB_RATING                
IMDB_NUMBEROFVOTES         
PLOT                       
US_MATURITY_RATING_STRING  
US_MATURITY_RATING         
GENRES 

In addition we added two fields that will help us with our fictional use case that are not derived from the  IMDb dataset

CREATION_TIMESTAMP         
PROMOTION

For the purpose of this workshop we will use the IMDb TT ID to provide a common identifier between the interactions data and the content metadata. Movielens provides its own identifier as well as a the IMDb TT ID (without the leading 'tt') in the 'links.csv' file. 


<div class="alert alert-block alert-warning">
<b>Note:</b>Your use of IMDb data is for the sole purpose of completing the AWS workshop and/or tutorial. Any use of IMDb data outside of the AWS workshop and/or tutorial requires a data license from IMDb. To obtain a data license, please contact: imdb-licensing-support@imdb.com. You will not (and will not allow a third party to) (i) use IMDb data, or any derivative works thereof, for any purpose; (ii) copy, sublicense, rent, sell, lease or otherwise transfer or distribute IMDb data or any portion thereof to any person or entity for any purpose not permitted within the workshop and/or tutorial; (iii) decompile, disassemble, or otherwise reverse engineer or attempt to reconstruct or discover any source code or underlying ideas or algorithms of IMDb data by any means whatsoever; or (iv) knowingly remove any product identification, copyright or other notices from IMDb data.</div>



Copy the subsection of IMDb item metadata to this repository.

In [102]:
 # copy IMDB data
!wget -P $imdb_dir https://d2peeor3oplhc6.cloudfront.net/personalize-immersionday-media/datasets/items.csv

--2024-04-27 21:58:01--  https://d2peeor3oplhc6.cloudfront.net/personalize-immersionday-media/datasets/items.csv
Resolving d2peeor3oplhc6.cloudfront.net (d2peeor3oplhc6.cloudfront.net)... 108.138.61.83, 108.138.61.141, 108.138.61.221, ...
Connecting to d2peeor3oplhc6.cloudfront.net (d2peeor3oplhc6.cloudfront.net)|108.138.61.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2320258 (2.2M) [text/csv]
Saving to: ‘poc_data/imdb/items.csv.4’

100%[======================================>] 2,320,258   --.-K/s   in 0.02s   

2024-04-27 21:58:01 (93.0 MB/s) - ‘poc_data/imdb/items.csv.4’ saved [2320258/2320258]



Next, load the IMDB `items.csv` file and take a look at the first rows. This file has information about the movie.

In [103]:
item_data = pd.read_csv(data_dir + '/imdb/items.csv', sep=',', dtype={'PROMOTION': "string"},index_col=0)
item_data.head(5)

,TITLE,YEAR,IMDB_RATING,IMDB_NUMBEROFVOTES,PLOT,US_MATURITY_RATING_STRING,US_MATURITY_RATING,GENRES,CREATION_TIMESTAMP,PROMOTION
ITEM_ID,,,,,,,,,,
tt0906325,Bagboy,2007,4,741,A teenager enters the competitive world of gro...,PG-13,13,Comedy,1167609600,false
tt0906665,Sukiyaki Western Django,2007,6,14582,A nameless gunfighter arrives in a town ripped...,R,17,Action|Western,1167609600,false
tt0907657,Once,2007,8,109787,A modern-day musical about a busker and an imm...,R,17,Drama|Music|Romance,1167609600,false
tt0910905,In the Electric Mist,2009,6,16592,A detective in post-Katrina New Orleans has a ...,R,17,Crime|Drama|Mystery|Thriller,1230768000,false
tt0910554,Frequently Asked Questions About Time Travel,2009,7,32051,"While drinking at their local pub, three socia...",PG-13,13,Comedy|Sci-Fi,1230768000,false


In [104]:
item_data.describe()

,YEAR,IMDB_RATING,IMDB_NUMBEROFVOTES,US_MATURITY_RATING,CREATION_TIMESTAMP
count,9298.000000,9298.000000,9.298000e+03,9298.000000,9.298000e+03
mean,1994.564207,6.639923,7.364282e+04,13.540977,7.751767e+08
std,18.575302,1.070418,1.472090e+05,4.647646,5.861922e+08
min,1902.000000,1.000000,1.900000e+01,0.000000,-2.145917e+09
25%,1988.000000,6.000000,7.561000e+03,8.000000,5.679936e+08
50%,1999.000000,7.000000,2.227600e+04,17.000000,9.151488e+08
75%,2008.000000,7.000000,7.404900e+04,17.000000,1.199146e+09
max,2018.000000,10.000000,2.303709e+06,18.000000,1.514765e+09


This does not really tell us much about the dataset, so we will explore a bit more and look at the raw information. We can see that genres often appear in groups. That is fine for us as Personalize supports this structure.

In [105]:
item_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9298 entries, tt0906325 to tt6911608
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   TITLE                      9298 non-null   object
 1   YEAR                       9298 non-null   int64 
 2   IMDB_RATING                9298 non-null   int64 
 3   IMDB_NUMBEROFVOTES         9298 non-null   int64 
 4   PLOT                       9298 non-null   object
 5   US_MATURITY_RATING_STRING  9298 non-null   object
 6   US_MATURITY_RATING         9298 non-null   int64 
 7   GENRES                     9298 non-null   object
 8   CREATION_TIMESTAMP         9298 non-null   int64 
 9   PROMOTION                  9298 non-null   string
dtypes: int64(5), object(4), string(1)
memory usage: 1.0+ MB


Now we have our Catalog of titles that our service offers. We also have some movies that the UnicornFlix marketing department would like to ensure are promoted in our recommendations. Amazon Personalize has a feature that allows you to promote items into recommendations, and set the balance of promoted items vs recommendations (we will cover this in detail in `03_Inference_Layer.ipynb`. Since we are in Las Vegas, lets create a promotion for movies about or set in Las Vegas. First we will find the movies in our catalog that feature or are set in/about Las Vegas and set the metadata field to true.

In [106]:
mask = item_data['PLOT'].str.contains('las vegas', case=False, na=False)
item_data.loc[mask, 'PROMOTION'] = 'true'
item_metadata = item_data
item_data[mask]

,TITLE,YEAR,IMDB_RATING,IMDB_NUMBEROFVOTES,PLOT,US_MATURITY_RATING_STRING,US_MATURITY_RATING,GENRES,CREATION_TIMESTAMP,PROMOTION
ITEM_ID,,,,,,,,,,
tt0066995,Diamonds Are Forever,1971,7,94484,A diamond smuggling investigation leads James ...,PG,8,Action|Adventure|Thriller,31536000,true
tt0475394,Smokin' Aces,2006,7,137003,When a Las Vegas performer-turned-snitch named...,R,17,Action|Comedy|Crime|Drama|Thriller,1136073600,true
tt0435705,Next,2007,6,147254,A Las Vegas magician who can see into the futu...,PG-13,13,Action|Sci-Fi|Thriller,1167609600,true
tt0120434,Vegas Vacation,1997,6,43985,In the fourth outing for the vacation franchis...,PG,8,Comedy,852076800,true
tt0120669,Fear and Loathing in Las Vegas,1998,8,257029,An oddball journalist and his psychopathic law...,R,17,Adventure|Comedy|Drama,883612800,true
...,...,...,...,...,...,...,...,...,...,...
tt1411697,The Hangover Part II,2011,6,459287,Two years after the bachelor party in Las Vega...,R,17,Comedy,1293840000,true
tt2231253,Wild Card,2015,6,50527,When a Las Vegas bodyguard with lethal skills ...,R,17,Action|Crime|Drama|Thriller,1420070400,true
tt2239832,Think Like a Man Too,2014,6,19623,All the couples are back for a wedding in Las ...,PG-13,13,Comedy|Romance,1388534400,true


Lets confirm that the changes we have made, have not introduced any null values:

In [107]:
item_data.isnull().sum()

TITLE                        0
YEAR                         0
IMDB_RATING                  0
IMDB_NUMBEROFVOTES           0
PLOT                         0
US_MATURITY_RATING_STRING    0
US_MATURITY_RATING           0
GENRES                       0
CREATION_TIMESTAMP           0
PROMOTION                    0
dtype: int64

Looks good, we currently have no null values.

That's it! At this point the item data is ready to go, and we just need to save it as a CSV file.

In [108]:
item_data.to_csv((data_dir+"/"+items_filename), index=True, float_format='%.0f')

## Prepare the Item Interactions data 

First, you will download the dataset from the [MovieLens project](https://grouplens.org/datasets/movielens/) website and unzip it in a new folder using the code below.

In [109]:
!cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!cd $data_dir && unzip -o ml-latest-small.zip 
dataset_dir = data_dir + "/ml-latest-small/"

--2024-04-27 21:58:01--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip.7’

100%[======================================>] 978,202     2.35MB/s   in 0.4s   

2024-04-27 21:58:02 (2.35 MB/s) - ‘ml-latest-small.zip.7’ saved [978202/978202]

Archive:  ml-latest-small.zip
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


Take a look at the data files you have downloaded.

In [110]:
!ls $dataset_dir

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


We can look at the README.txt file and licensing, do not skip over usage license!

In [111]:
!pygmentize $data_dir/ml-latest-small/README.txt

Summary

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for down

The primary data we are interested in for a recommendation use case is the actual interactions that the users had with the titles(items). 

Open the `ratings.csv` file and take a look at the some rows from throughout the dataset.

In [112]:
interaction_data = pd.read_csv(dataset_dir + '/ratings.csv', sep=',', dtype={'userId': "int64", 'movieId': "str"})
interaction_data.sample(10)

,userId,movieId,rating,timestamp
5101,33,1027,3.0,939716819
5950,42,459,3.0,996220584
44748,298,3698,2.5,1450452365
91495,593,53464,2.0,1181007732
92437,597,2100,4.0,941641110
28592,199,296,4.0,940372462
9096,62,118696,4.0,1523111252
9971,65,89774,4.0,1494767219
59897,387,5013,4.0,1094876931
68956,448,2018,4.0,1019126704


To use Amazon Personalize, you need to save timestamps in Unix Epoch format.

Lets validate that the timestamp is actually in a Unix Epoch format by converting it into a more easily understood time/date format

In [113]:
arb_time_stamp = interaction_data.iloc[50]['timestamp']
print('timestamp')
print(arb_time_stamp)
print()
print('Date & Time')
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

timestamp
964982681

Date & Time
2000-07-30 18:44:41


We will do some general summarization and inspection of the data to ensure that it will be helpful for Amazon Personalize

In [114]:
interaction_data.isnull().any()

userId       False
movieId      False
rating       False
timestamp    False
dtype: bool

In [115]:
interaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  object 
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 3.1+ MB


What you can see is that the Movielens dataset is that this dataset contains a userId, a movieId, the rating that the user gave the movie and the time the made this interaction. For the purposes of our fictional service UnicornFlix this data will stand in for our applications interaction data, which would actually be the click stream data of the titles that were watched, in the order they watched them.

### Convert the Interactions Data

The interaction data generally is acquired from anaytics or CDP platforms that can identify individual interactions with content/items within a platform. 

We need to do a few things to get this dataset ready to subsitute for our services interaction data.

First off, the movieId is a unique identifier provided by Movielens for each title. However as we saw above IMDb has a much richer set of metadata about the content catalog. In order to use the IMDb data we will need to use a common  identifier between our items and our interactions dataset, which is the IMDb imdbId. To do this Movielens provides the 'links.csv' file which helps convert between the two identifiers.

In [116]:
links = pd.read_csv(dataset_dir + '/links.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "str", 'imdbId': "str", 'tmdbId': "str"})
pd.set_option('display.max_rows', 25)
links['imdbId'] = 'tt' + links['imdbId'].astype(object)
links

,movieId,imdbId
0,1,tt0114709
1,2,tt0113497
2,3,tt0113228
3,4,tt0114885
4,5,tt0113041
...,...,...
9737,193581,tt5476944
9738,193583,tt5914996
9739,193585,tt6397426
9740,193587,tt8391976


As you can see this provides a method to identify what the IMDb id is for every title in our interactions dataset, now we will convert the ratings.csv data to utilize the IMDb ID.

In [117]:
imdb_data = interaction_data.merge(links, on='movieId')
imdb_data.drop(columns='movieId')

,userId,rating,timestamp,imdbId
0,1,4.0,964982703,tt0114709
1,1,4.0,964981247,tt0113228
2,1,4.0,964982224,tt0113277
3,1,5.0,964983815,tt0114369
4,1,5.0,964982931,tt0114814
...,...,...,...,...
100831,610,4.0,1493848402,tt4972582
100832,610,5.0,1493850091,tt4425200
100833,610,5.0,1494273047,tt5052448
100834,610,5.0,1493846352,tt3315342


Now we have a interactions dataset that matches our item catalog dataset. 

### Simulating a interaction dataset 

We are going to make one more modification to make the MoviesLens dataset more like the analytics data that a video streaming service would see in their interactions. MoviesLens is an explicit movie rating dataset, which means users are presented a movie and asked to give it a rating. For recommendation systems/personalization, the industry has moved on to using more implicit data. This is due to many reasons including low numbers of customers rating titles and customers tastes changing over time. Some of the benefits of implicit interaction data is that it is the actual behavior of all users and changes over time as their viewing behavior changes.

To convert the explicit interaction MovieLens ratings dataset into our fictional streaming service UnicornFlix's implicit dataset we are going to create a synthetic dataset using the ratings in MovieLens. 

- Implicit interactions are inherently positive interactions so we will be dropping any rating that is below 2 stars 
- Ratings of 2 and 3 stars are neutral to slightly positive, we are going to create synthetic "Click" events to simulate a viewer clicking on a title in the UnicornFlix app
- Ratings of 4 and 5 are overwhelmingly positive, we will use these to create synthetic "Watch" and "Click" events to simulate a viewer both clicking on a title and watching at least 80% of a title in the UnicornFlix app

<div class="alert alert-block alert-warning">
<b>Note:</b> This will be directionaly accurate, but is not a good substitute for actual temporal based interaction data, the order that viewers rated movies on the MovieLens website is not as good as the order of interactions on an actual Video On Demand Streaming app. For more information about the importance of the temporal interaction data see
https://www.amazon.science/publications/temporal-contextual-recommendation-in-real-time
</div> 

In [118]:
watched_df = imdb_data.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'imdbId', 'timestamp']]
watched_df['EVENT_TYPE']='Watch'
watched_df.head()

,userId,imdbId,timestamp,EVENT_TYPE
0,1,tt0114709,964982703,Watch
1,1,tt0113228,964981247,Watch
2,1,tt0113277,964982224,Watch
3,1,tt0114369,964983815,Watch
4,1,tt0114814,964982931,Watch


In [119]:
clicked_df = imdb_data.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'imdbId', 'timestamp']]
clicked_df['EVENT_TYPE']='Click'
clicked_df.head()

,userId,imdbId,timestamp,EVENT_TYPE
0,1,tt0114709,964982703,Click
1,1,tt0113228,964981247,Click
2,1,tt0113277,964982224,Click
3,1,tt0114369,964983815,Click
4,1,tt0114814,964982931,Click


In [120]:
interactions_df = clicked_df.copy()

interactions_df = pd.concat([interactions_df, watched_df])
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')

Lets look at what the new dataset looks like and ensure that the data reflects our fictional streaming services streaming analytics data

In [121]:
interactions_df

,userId,imdbId,timestamp,EVENT_TYPE
66679,429,tt0112679,828124615,Watch
66681,429,tt0109676,828124615,Click
66719,429,tt0101414,828124615,Watch
66718,429,tt0096895,828124615,Watch
66717,429,tt0099348,828124615,Watch
...,...,...,...,...
81477,514,tt3778644,1537674946,Click
81336,514,tt0076729,1537757040,Click
81335,514,tt0081529,1537757059,Click
81092,514,tt0109508,1537799250,Watch


 Amazon Personalize has default column names for users, items, and timestamp. These default column names are `USER_ID`, `ITEM_ID`, `TIMESTAMP` and `EVENT_VALUE` for the [VIDEO_ON_DEMAND domain dataset](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO-ON-DEMAND-datasets-and-schemas.html). The final modification to the dataset is to replace the existing column headers with the default headers.

In [122]:
interactions_df.rename(columns = {'userId':'USER_ID', 'imdbId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

We'll be using a subset of the IMDB dataset for this workshop that has been cleaned to remove movies that don't have valid values for the metadata we are using in out ITEMs dataset (we'll work with this more in the net section), so we'll need to make sure we don't have any interactions that have IMDB movie ids that are not in our subset of the IMDB data set.

In [123]:
movies = pd.read_csv(data_dir + '/imdb' + '/items.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "str", 'imdbId': "str", 'tmdbId': "str"})
pd.set_option('display.max_rows', 25)

Next, let's compare the number of `ITEM_ID` unique keys in the IMDB data to the `ITEM_ID` unique keys in the interactions.  They should be the same.

In [124]:
movies.nunique(axis=0)

ITEM_ID    9298
TITLE      8981
dtype: int64

The number of unique ITEM_IDs are not the same in the IMDB data and the interactions data, so we'll clean out the data points with ITEM_IDs that do not have item metadata from the interactions dataset.

In [125]:
interactions_df = interactions_df.merge(movies, on='ITEM_ID')
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157395 entries, 0 to 157394
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   USER_ID     157395 non-null  int64 
 1   ITEM_ID     157395 non-null  object
 2   TIMESTAMP   157395 non-null  int64 
 3   EVENT_TYPE  157395 non-null  object
 4   TITLE       157395 non-null  object
dtypes: int64(2), object(3)
memory usage: 6.0+ MB


We will also drop the `TITLE` column as it is not required in the interactions dataset.

In [126]:
interactions_df = interactions_df.drop(columns=['TITLE'])
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157395 entries, 0 to 157394
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   USER_ID     157395 non-null  int64 
 1   ITEM_ID     157395 non-null  object
 2   TIMESTAMP   157395 non-null  int64 
 3   EVENT_TYPE  157395 non-null  object
dtypes: int64(2), object(2)
memory usage: 4.8+ MB


That's it! At this point the data is ready to go, and we just need to save it as a CSV file.

In [127]:
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## Prepare the User Metadata <a class="anchor" id="prepare_users"></a>
[Back to top](#top)

The dataset does not have any user metadata so we will create a synthetic metadata field that would be an example of the type of user metadata UnicornFlix may have in their CRM/Subcriber management system. This data will be used both for training of the models, but also can be used for inference filters, which will be covered in a later notebook.

In [128]:
# get all unique user ids from the interaction dataset

user_ids = interactions_df['USER_ID'].unique()
user_data = pd.DataFrame()
user_data["USER_ID"]= user_ids
user_data

,USER_ID
0,429
1,107
2,191
3,99
4,54
...,...
605,77
606,25
607,596
608,184


### Adding User Metadata

The current dataset does not contain additiona user information. For this example, we'll randomly assign a membership level. For Ad Supported models this could indicate premium vs ad supported.

<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: This is a synthetic dataset and since it is randomly assigned, will be of little value to our mode, in a real world scenario this data would be accurate to the user data.
</div>

NOTE: This is a synthetic dataset and since it is randomly assigned, will be of little value to our mode, in a real world scenario this data would be accurate to the user data.

In [129]:
possible_membership_levels = ['silver', 'gold']
random = np.random.choice(possible_membership_levels, len(user_data.index), p=[0.5, 0.5])
user_data["MEMBERLEVEL"] = random
user_data

,USER_ID,MEMBERLEVEL
0,429,gold
1,107,gold
2,191,silver
3,99,gold
4,54,silver
...,...,...
605,77,gold
606,25,gold
607,596,gold
608,184,gold


That's it! At this point the data is ready to go, and we just need to save it as a CSV file.

In [130]:
# Saving the data as a CSV file
user_data.to_csv((data_dir+"/"+users_filename), index=False, float_format='%.0f')

In [131]:
# Creating Amazon Personalize Resources and Importing data <a class="anchor" id="import"></a>

## Configure an S3 bucket and an IAM  role <a class="anchor" id="bucket_role"></a>

So far, we have downloaded, manipulated, and saved the data onto the Amazon EBS instance attached to the instance running this Jupyter notebook.  

By default, the Personalize service does not have permission to access the data we upload into  S3 buckets in our account. In order to grant access to the Amazon Personalize service to interact with our S3 Buckets, we need to set a Bucket Policy and create an IAM role that the Amazon Personalize service will assume. If you are running this notebook without also running the Pretrained cloud formation template in the root folder then you will need to grant this Notebook substantial permissions in order to be able to run the code correctly.

If you are running this with the pretrained cloud formation template `PersonalizeIDPretrained.yaml` then the notebook will not need that level of permissioning and will obtain the permissions from the bucket created as part of the automation process for demonstration.

Use the metadata stored on the instance underlying this Amazon SageMaker notebook, to determine the region it is operating in. If you are using a Jupyter notebook outside of Amazon SageMaker, simply define the region as a string below. The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources we have been creating so far.

First, let us get the current notebook region.

In [132]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]

# To use a different region use:
# region = <your_region>

print('region:', region)

region: us-east-1


Amazon S3 bucket names are globally unique. To create a unique bucket name, the code below will append the string `personalize-poc-media` to your AWS account number. Then it creates a bucket with this name in the region discovered in the previous cell. Note if you have already created a bucket as part of the cloud formation automation then the cell below will return information on that previously created bucket.

In [133]:
# Configure the SDK to SSM:
ssm = boto3.client('ssm')
s3 = boto3.client('s3')
bucket_found = False
try:
    personalizes3bucket = ssm.get_parameter(Name='/cloudformation/personalize-s3-bucket', WithDecryption=False)
    bucket_name = personalizes3bucket['Parameter']['Value']
    print('Bucket created as part of cloud formation template found')
    print('bucket_name:', bucket_name)
    bucket_found=True
except:
    
    account_id = boto3.client('sts').get_caller_identity().get('Account')
    bucket_name = account_id + "-" + region + "-" + "personalize-poc-media"

    #getting existing buckets in the account
    response = s3.list_buckets()

    if bucket_name in [x['Name'] for x in response['Buckets']]:
        print("The bucket already exists.")
    else:
        if region == "us-east-1":
            bucket_responese = s3.create_bucket(Bucket=bucket_name)
        else:
            bucket_responese = s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
                )
    print('bucket_name:', bucket_name)

Bucket created as part of cloud formation template found
bucket_name: test-stack-3-mlopsstack-1ksuxvta-personalizebucket-evtxjufeydvb


Amazon Personalize needs to be able to read the contents of your S3 bucket. The policy which enables personalize to access the contents of the S3 bucket is below.

```python
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}
```

This S3 bucket policy allows Amazon Personalize to be able to read the contents of your S3 bucket. The code below creates and adds the policy to the bucket it created in the last step. If the bucket was created as part of the automation script then it merely shows the bucket that it created.

In [134]:
if bucket_found:
    bucket_current_policy = s3.get_bucket_policy(Bucket=bucket_name)['Policy']
    print("Policy for bucket created as part of cloud formation template:")
    print(json.loads(bucket_current_policy))
else:
    policy = {
        "Version": "2012-10-17",
        "Id": "PersonalizeS3BucketAccessPolicy",
        "Statement": [
            {
                "Sid": "PersonalizeS3BucketAccessPolicy",
                "Effect": "Allow",
                "Principal": {
                    "Service": "personalize.amazonaws.com"
                },
                "Action": [
                    "s3:GetObject",
                    "s3:ListBucket",
                    "s3:PutObject"
                ],
                "Resource": [
                    "arn:aws:s3:::{}".format(bucket_name),
                    "arn:aws:s3:::{}/*".format(bucket_name)
                ]
            }
        ]
    }

    bucket_current_policy = None

    try:
        bucket_current_policy = s3.get_bucket_policy(Bucket=bucket_name)['Policy']

    except s3.exceptions.from_code('NoSuchBucketPolicy') as e:    
        print("There is no current Bucket Policy for bucket " + bucket_name)

    except Exception as e: 
        raise(e)

    if (bucket_current_policy and policy == json.loads(bucket_current_policy)):
        print ("The policy is already associated with the S3 Bucket.")
    else:
        print ("Adding the policy to the bucket.")
        print(s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy)))

Policy for bucket created as part of cloud formation template:
{'Version': '2012-10-17', 'Id': 'PersonalizeS3BucketAccessPolicy', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'personalize.amazonaws.com'}, 'Action': ['s3:GetObject', 's3:PutObject', 's3:ListBucket'], 'Resource': ['arn:aws:s3:::test-stack-3-mlopsstack-1ksuxvta-personalizebucket-evtxjufeydvb', 'arn:aws:s3:::test-stack-3-mlopsstack-1ksuxvta-personalizebucket-evtxjufeydvb/*']}]}


### Create an IAM role

Amazon Personalize also needs the ability to assume roles in AWS in order to have the permissions to execute certain tasks. Let's create an IAM role and attach the required policies to it. The code below attaches broad policies. You should use [more restrictive, least-privilege policies for production applications](https://docs.aws.amazon.com/IAM/latest/UserGuide/best-practices.html#grant-least-privilege). If you have run the cloud formation template the role will have been automatically created and we will simply obtain that one.

In [135]:
if bucket_found:
    print("Cloud formation template used - skipping creation of IAM role and needed policies for Personalize as they were already created as part of the automation script")
    role_arn_info = ssm.get_parameter(Name='/cloudformation/personalize-iam-role-arn', WithDecryption=False)
    role_arn = role_arn_info['Parameter']['Value']
    role_name = role_arn.split('/')[1]
else:
    iam = boto3.client("iam")

    role_name = account_id+"-PersonalizeS3-Immersion-Day"
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                "Service": "personalize.amazonaws.com"
              },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    # Create policy

    s3_access_policy_document = {
        "Version": "2012-10-17",
        "Statement": {
                "Sid" : "myStatement" ,
                "Effect": "Allow",
                "Resource": [
                    "arn:aws:s3:::{}".format(bucket_name),
                    "arn:aws:s3:::{}/*".format(bucket_name)
                ],
                "Action": "s3:*"
            }
    }

    try: 

        policy_response = iam.create_policy(
            PolicyName='restrictedS3Access',
            PolicyDocument=json.dumps(s3_access_policy_document),
            Description='Restricts access to only workshop S3 bucket'
        )

        s3_access_policy_arn = policy_response['Policy']['Arn']

        print ("s3_access_policy_arn:{}".format(s3_access_policy_arn))
    except:
        s3_access_policy_arn = 'arn:aws:iam::{}:policy/restrictedS3Access'.format(account_id)
        print ('The policy {} already exists.'.format(s3_access_policy_arn))
        print ('Using the existing policy')


    try:
        create_role_response = iam.create_role(
            RoleName = role_name,
            AssumeRolePolicyDocument = json.dumps(assume_role_policy_document),
        );
        role_arn = create_role_response["Role"]["Arn"]

        print ("10s pause to allow role to be fully consistent.")
        time.sleep(10)

    except iam.exceptions.EntityAlreadyExistsException as e:
        print('Warning: role already exists:', e)
        role_arn = iam.get_role(
            RoleName = role_name
        )["Role"]["Arn"];

    print('IAM Role: {}\n'.format(role_arn))

    # Attach the policy if it is not previously attached:
    if (s3_access_policy_arn in [ x['PolicyArn'] for x in iam.list_attached_role_policies( RoleName = role_name)['AttachedPolicies']]):
        print ('The policy {} is already attached to this role.'.format(s3_access_policy_arn))
    else:
        print ("Attaching the role_policy: {}".format(s3_access_policy_arn))
        attach_response = iam.attach_role_policy(
            RoleName = role_name,
            PolicyArn = s3_access_policy_arn
        );
        print ("30s pause to allow role to be fully consistent.")
        time.sleep(30)
        print('Done.')

The policy arn:aws:iam::381491864570:policy/restrictedS3Access already exists.
Using the existing policy
IAM Role: arn:aws:iam::381491864570:role/381491864570-PersonalizeS3-Immersion-Day

The policy arn:aws:iam::381491864570:policy/restrictedS3Access is already attached to this role.


### Upload data to S3

Now that your Amazon S3 bucket has been created, upload the CSV files of our 3 datasets (Item, Interaction and User).


<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: We will cover how to import real-time data in a future notebook..
</div>


In [136]:
interactions_file_path = data_dir + "/" + interactions_filename

try:
    s3.get_object(
        Bucket=bucket_name,
        Key="media/interactions.csv",
    )
    print("{} already exists in the bucket {}".format("media/interactions.csv", bucket_name))
except s3.exceptions.NoSuchKey:
    # Uploading the file if it does not already exist
    boto3.Session().resource('s3').Bucket(bucket_name).Object("media/interactions.csv").upload_file(interactions_file_path)
    print("File {} uploaded to bucket {}".format("media/interactions.csv", bucket_name))

items_file_path = data_dir + "/" + items_filename
try:
    s3.get_object(
        Bucket=bucket_name,
        Key="media/items.csv",
    )
    print("{} already exists in the bucket {}".format("media/items.csv", bucket_name))
except s3.exceptions.NoSuchKey:
    # Uploading the file if it does not already exist     
    boto3.Session().resource('s3').Bucket(bucket_name).Object("media/items.csv").upload_file(items_file_path)
    print("File {} uploaded to bucket {}".format("media/items.csv", bucket_name))

users_file_path = data_dir + "/" + users_filename
try:
    s3.get_object(
        Bucket=bucket_name,
        Key="media/users.csv",
    )
    print("{} already exists in the bucket {}".format("media/users.csv", bucket_name))
except s3.exceptions.NoSuchKey:
    # Uploading the file if it does not already exist
    boto3.Session().resource('s3').Bucket(bucket_name).Object("media/users.csv").upload_file(users_file_path)
    print("File {} uploaded to bucket {}".format("media/users.csv", bucket_name))

media/interactions.csv already exists in the bucket test-stack-3-mlopsstack-1ksuxvta-personalizebucket-evtxjufeydvb
media/items.csv already exists in the bucket test-stack-3-mlopsstack-1ksuxvta-personalizebucket-evtxjufeydvb
media/users.csv already exists in the bucket test-stack-3-mlopsstack-1ksuxvta-personalizebucket-evtxjufeydvb


We will use this data in the next labs. In order to use this data we will store these variables so subsequent notebooks can use this data. 

In [137]:
%store dataset_dir
%store data_dir
%store interactions_filename
%store items_filename
%store users_filename
%store workshop_rerank_solution_name
%store workshop_rerank_campaign_name
%store recommender_more_like_x_name
%store recommender_top_picks_for_you_name
%store region
%store account_id
%store workshop_dataset_group_name
%store interactions_schema_name
%store items_schema_name
%store users_schema_name
%store interactions_dataset_name
%store items_dataset_name
%store users_dataset_name
%store items_import_job_name
%store interactions_import_job_name
%store users_import_job_name
%store bucket_name
%store role_name
%store role_arn

Stored 'dataset_dir' (str)
Stored 'data_dir' (str)
Stored 'interactions_filename' (str)
Stored 'items_filename' (str)
Stored 'users_filename' (str)
Stored 'workshop_rerank_solution_name' (str)
Stored 'workshop_rerank_campaign_name' (str)
Stored 'recommender_more_like_x_name' (str)
Stored 'recommender_top_picks_for_you_name' (str)
Stored 'region' (str)
Stored 'account_id' (str)
Stored 'workshop_dataset_group_name' (str)
Stored 'interactions_schema_name' (str)
Stored 'items_schema_name' (str)
Stored 'users_schema_name' (str)
Stored 'interactions_dataset_name' (str)
Stored 'items_dataset_name' (str)
Stored 'users_dataset_name' (str)
Stored 'items_import_job_name' (str)
Stored 'interactions_import_job_name' (str)
Stored 'users_import_job_name' (str)
Stored 'bucket_name' (str)
Stored 'role_name' (str)
Stored 'role_arn' (str)


[Go to the next notebook `Media_02_Data_Import`](Media_02_Data_Import.ipynb) to continue.